## Model sici neurnonwej do rozpoznawania enchancerów

In [3]:
import pandas as pd
import gzip
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [5]:
# tworzenie słownika zawierający wszystkie k-mery danej długości
def create_kmer_dict(k):
    bases = ['A', 'C', 'G', 'T']
    kmer_dict = {}
    for i in range(4**k):
        kmer = ''.join([bases[(i // (4**j)) % 4] for j in range(k)])
        rev_comp = ''.join([{'A':'T', 'C':'G', 'G':'C', 'T':'A'}[base] for base in kmer[::-1]])
        if kmer not in kmer_dict and rev_comp not in kmer_dict:
            kmer_dict[kmer] = 0
    return kmer_dict

In [6]:
# tworzenie słownika zliczającego ilość k-merów danej sekwencji uwzględniająć odwrotną komplementarność
def seq_kmers_dict(dna_sequence, k, kmers_dicts):
    number_of_kmer = len(dna_sequence) - k + 1
    for i in range(len(dna_sequence) - k + 1):
        kmer = dna_sequence[i:i+k]
        rc_kmer = reverse_complement(kmer)
        if kmer in kmers_dicts:
            kmers_dicts[kmer] += 1/number_of_kmer
        elif rc_kmer in kmers_dicts:
            kmers_dicts[rc_kmer] += 1/number_of_kmer
    return kmers_dicts

def reverse_complement(dna_sequence):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    rc_sequence = ""
    for base in reversed(dna_sequence):
        rc_sequence += complement[base]
    return rc_sequence

In [7]:
# Wczytanie sekwencji dna
df_fasta = pd.DataFrame(columns=['ID', 'sequence'])
with gzip.open("GRCh37.primary_assembly.genome.fa.gz", "rt") as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        sequence = record.seq
        df_fasta = pd.concat([df_fasta, pd.DataFrame.from_records([{'ID':identifier, 'sequence':sequence}])], ignore_index=True)

# Model 5-mer

In [8]:
kmer_dict = create_kmer_dict(5)

In [6]:
#zbiór pozytywny
positive_seq = list()
with open('positive.bed') as f:
    for line in f:
        name, start, stop, tmp = line.split()
        positive_seq.append(str(list(df_fasta.loc[df_fasta['ID'] == name, 'sequence'])[0][int(start)-1:int(stop)]))

positive_seq = [x for x in positive_seq if 'N' not in x ]
positive_set = [seq_kmers_dict(x, 5, kmer_dict.copy()) for x in positive_seq]
train_x_positive = pd.DataFrame(positive_set)

In [7]:
#zbiór negatywny
negative_seq = list()
with open('negative.bed') as f:
    for line in f:
        name, start, stop = line.split()
        negative_seq.append(str(list(df_fasta.loc[df_fasta['ID'] == name, 'sequence'])[0][int(start)-1:int(stop)]))

negative_seq = [x for x in negative_seq if 'N' not in x ]
negative_set = [seq_kmers_dict(x, 5, kmer_dict.copy()) for x in negative_seq]
train_x_negative = pd.DataFrame(negative_set)

In [8]:
#stworzenie zbioru 
train_x = pd.concat([train_x_positive, train_x_negative])
train_y = pd.DataFrame([1]*41513 + [0]*72389)

# Model 5-mer z podziałem na zbiór treningowy i testowy

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

In [10]:
tf.random.set_seed(4321)
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    #tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    #tf.keras.layers.Dense(70, activation=tf.nn.relu),
                                    #tf.keras.layers.Dense(70, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(8, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation='sigmoid')])
model2.compile(optimizer='adam', loss = tf.keras.losses.binary_crossentropy, metrics=[tf.keras.metrics.Recall()])

In [11]:
model2.fit(x_train, y_train, epochs=70)

Epoch 1/70
2492/2492 [==============================] - 7s 2ms/step - loss: 0.5673 - recall: 0.4329
Epoch 2/70
2492/2492 [==============================] - 6s 2ms/step - loss: 0.5422 - recall: 0.5311
Epoch 3/70
2492/2492 [==============================] - 6s 2ms/step - loss: 0.5375 - recall: 0.5399
Epoch 4/70
2492/2492 [==============================] - 6s 3ms/step - loss: 0.5348 - recall: 0.5381
Epoch 5/70
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5318 - recall: 0.5401
Epoch 6/70
2492/2492 [==============================] - 6s 2ms/step - loss: 0.5294 - recall: 0.5386
Epoch 7/70
2492/2492 [==============================] - 6s 2ms/step - loss: 0.5271 - recall: 0.5455
Epoch 8/70
2492/2492 [==============================] - 6s 2ms/step - loss: 0.5237 - recall: 0.5479
Epoch 9/70
2492/2492 [==============================] - 6s 2ms/step - loss: 0.5212 - recall: 0.5501
Epoch 10/70
2492/2492 [==============================] - 6s 2ms/step - loss: 0.5188 - recall: 0.5496

In [12]:
model2.save('model5kmer_train_test')

INFO:tensorflow:Assets written to: model5kmer_train_test\assets


INFO:tensorflow:Assets written to: model5kmer_train_test\assets


## Metryki dla zbioru treningowego i testowego

In [13]:
pred = model2.predict(x_train)
enchancer = (pred > 0.5).astype(int)
enchcer_true = y_train.to_numpy()

2492/2492 [==============================] - 3s 1ms/step


In [14]:
print(accuracy_score(enchcer_true, enchancer ))
print(recall_score(enchcer_true, enchancer ))
print(precision_score(enchcer_true, enchancer ))
print(f1_score(enchcer_true, enchancer ))

0.8701383401688176
0.7585624116165971
0.8676766481240383
0.809458962090541


In [15]:
pred = model2.predict(x_test)
enchancer = (pred > 0.5).astype(int)
enchcer_true = y_test.to_numpy()

1068/1068 [==============================] - 1s 1ms/step


In [16]:
print(accuracy_score(enchcer_true, enchancer ))
print(recall_score(enchcer_true, enchancer ))
print(precision_score(enchcer_true, enchancer ))
print(f1_score(enchcer_true, enchancer ))

0.7101050598460683
0.5329872204472843
0.621785314945956
0.5739721314295544


# Drugi model

In [42]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

In [43]:
tf.random.set_seed(4321)
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    #tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    #tf.keras.layers.Dense(70, activation=tf.nn.relu),
                                    #tf.keras.layers.Dense(70, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(8, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation='sigmoid')])
model2.compile(optimizer='adam', loss = tf.keras.losses.binary_crossentropy, metrics=[tf.keras.metrics.Recall()])

In [44]:
model2.fit(x_train, y_train, epochs=100)

Epoch 1/100
2492/2492 [==============================] - 8s 3ms/step - loss: 0.5674 - recall: 0.4461
Epoch 2/100
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5435 - recall: 0.5290
Epoch 3/100
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5387 - recall: 0.5380
Epoch 4/100
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5358 - recall: 0.5355
Epoch 5/100
2492/2492 [==============================] - 6s 3ms/step - loss: 0.5326 - recall: 0.5435
Epoch 6/100
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5304 - recall: 0.5415
Epoch 7/100
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5279 - recall: 0.5456
Epoch 8/100
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5241 - recall: 0.5497
Epoch 9/100
2492/2492 [==============================] - 7s 3ms/step - loss: 0.5220 - recall: 0.5513
Epoch 10/100
2492/2492 [==============================] - 6s 3ms/step - loss: 0.5193 - reca

## Metryki dla zbioru treningowego i testowego

In [45]:
pred = model2.predict(x_train)
enchancer = (pred > 0.5).astype(int)
enchcer_true = y_train.to_numpy()

2492/2492 [==============================] - 3s 1ms/step


In [46]:
print(accuracy_score(enchcer_true, enchancer ))
print(recall_score(enchcer_true, enchancer ))
print(precision_score(enchcer_true, enchancer ))
print(f1_score(enchcer_true, enchancer ))

0.9166948865560447
0.8254751146828545
0.937997256515775
0.8781463271446394


In [47]:
pred = model2.predict(x_test)
enchancer = (pred > 0.5).astype(int)
enchcer_true = y_test.to_numpy()

1068/1068 [==============================] - 1s 1ms/step


In [48]:
print(accuracy_score(enchcer_true, enchancer ))
print(recall_score(enchcer_true, enchancer ))
print(precision_score(enchcer_true, enchancer ))
print(f1_score(enchcer_true, enchancer ))

0.69237072371309
0.5060702875399361
0.5941485371342836
0.546583850931677


# predykcja z sekwencji dna

In [9]:
dna_seq = "AAAGACCCCTTCACTGACATGGTGAATACACTGTGTAGGTCCAGAGACAGTGATGACAAACTAAGTTTCACAGACTTTGAATTTGGGAAGGCCAGTGAGATATCGCAGCCAAGATACCCCAGAGGCAGTGGAGCAGGGTGAATAAAGTCTCACTAGAGGGTCAGATTTGGCAAATGCGTGGGTTGCAGTTGAAGCCACAGATGTGAATTGCCGTCCAGTAAGTGTAGGGGGAGTAAAGATCTAGGATAAGGAGAGAAAGAGAAACTGGCACTGACAGAAAGAAACAGGAAGAGAAAAAGCTGACAGAAACTACATGGAACCTACCACACTCAAGGGAGGTTGGCTAATGGTGTCACGTGATGCCAGGACACCTGGCGAGTTAGGAACCTGGGAAGAGTTATTTTGTGACCTGGGCTGACAGTTTTATTAAAAGGGACAAGTGAGTGGCCAGAGTGTGAGGGGCCGAGGCAGGCCTGGCAGGAATGAGGGTGGGCTGTTAAGGTGATCTGGCTGGGCCACAGGGGCGGGGGAGAGAGGCGTTGGTTTGTCATTGTTCTTGAATTAAGCATGTTGAGTTGCCAACTACAAGATGCTA"

In [10]:
x = pd.DataFrame(seq_kmers_dict(dna_seq, 5, kmer_dict.copy()), index=[0])

In [11]:
# szansa że sekwencja jest enchancerem
model2.predict(x)[0][0]

1/1 [==============================] - 0s 90ms/step


0.998462

# WebService

In [4]:
model2 = tf.keras.models.load_model('model5kmer_train_test')

## średnie długości 

In [12]:
#obliczanie średnie długości
enchancer_lengths = pd.DataFrame(columns=['name', 'lenght'])
with open('positive_all.bed') as f:
    for line in f:
        name, start, stop, tmp = line.split()
        new_row = {'name': name, 'lenght': int(stop)-int(start)+1}
        enchancer_lengths.loc[len(enchancer_lengths)] = new_row

In [13]:
srednie = enchancer_lengths.groupby('name')['lenght'].mean()
srednie = srednie.round().astype(int)
srednie = srednie[0:22]

In [45]:
srednie

name
chr1     1730
chr10    1773
chr11    1730
chr12    1747
chr13    1677
chr14    1907
chr15    1831
chr16    1761
chr17    1792
chr18    1682
chr19    1468
chr2     1786
chr20    1749
chr21    1836
chr22    1875
chr3     1719
chr4     1675
chr5     1767
chr6     1791
chr7     1636
chr8     1804
chr9     1767
Name: lenght, dtype: int32

In [46]:
srednie.index

Index(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16',
       'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3',
       'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9'],
      dtype='object', name='name')

## predykcja

In [14]:
web_data = pd.DataFrame(columns=['name', 'mean_lenght', 'prediction', 'prediction_percent'])

for name in srednie.index:
    print(name)
    seq_number = len(list(df_fasta.loc[df_fasta['ID'] == name, 'sequence'])[0])//srednie[name]
    positions = [[srednie[name]//2*i, srednie[name]//2*i + srednie[name]]  for i in range(seq_number)]
    
    enchancer_seq = list()
    for seq in positions:
        start = seq[0]
        stop = seq[1]
        enchancer_seq.append(str(list(df_fasta.loc[df_fasta['ID'] == name, 'sequence'])[0][int(start)-1:int(stop)]))
        
        
    false_ench = seq_kmers_dict("AAAAA", 5, kmer_dict.copy())
    enchancer_data = [seq_kmers_dict(x, 5, kmer_dict.copy()) if 'N' not in x else false_ench for x in enchancer_seq]
    enchancer_data = pd.DataFrame(enchancer_data)
    
    
    prediction_percent = model2.predict(enchancer_data)
    prediction_0_1 = (prediction_percent > 0.5).astype(int)
    prediction_percent = [x[0] for x in prediction_percent]
    prediction_0_1 = [x[0] for x in prediction_0_1]

    
    new_row = {'name': name, 'mean_lenght': srednie[name], 'prediction': prediction_0_1, 'prediction_percent': prediction_percent }
    web_data.loc[len(web_data)] = new_row

    
web_data = web_data.set_index('name', drop=True)    
web_data.to_csv('web_data.csv') 

chr1
4503/4503 [==============================] - 6s 1ms/step
chr10
2389/2389 [==============================] - 4s 2ms/step
chr11
2439/2439 [==============================] - 4s 2ms/step
chr12
2395/2395 [==============================] - 3s 1ms/step
chr13
2147/2147 [==============================] - 3s 1ms/step
chr14
1760/1760 [==============================] - 3s 2ms/step
chr15
1750/1750 [==============================] - 3s 2ms/step
chr16
1604/1604 [==============================] - 3s 2ms/step
chr17
1416/1416 [==============================] - 2s 2ms/step
chr18
1451/1451 [==============================] - 2s 2ms/step
chr19
1259/1259 [==============================] - 2s 1ms/step
chr2
4256/4256 [==============================] - 5s 1ms/step
chr20
1127/1127 [==============================] - 1s 1ms/step
chr21
820/820 [==============================] - 1s 1ms/step
chr22
856/856 [==============================] - 1s 1ms/step
chr3
3600/3600 [==============================] - 4s 1ms/step

## wczytanie i zamiany

In [ ]:
tmp1 = pd.read_csv('web_data.csv')  

In [ ]:
[int(x) for x in tmp1.loc[tmp1['name'] == 'chr1']['prediction'][0][1:-1].split(',')]

In [61]:
len(df_fasta.loc[df_fasta['ID'] == 'chr1']['sequence'][0])/2000

124625.3105